In [1]:
from vivarium import Artifact, InteractiveContext
import pandas as pd, numpy as np, os

In [2]:
! pip list | grep vivarium

# [software verion + hash . date]

vivarium                  3.6.6
vivarium_build_utils      2.1.2
vivarium_cluster_tools    2.2.2
vivarium_dependencies     1.0.4
vivarium_gates_mncnh      25.1.dev10+g18b5f521f.d20260120 /mnt/share/homes/zmbc/src/vivarium_gates_mncnh
vivarium_public_health    4.13.19
vivarium_testing_utils    0.3.1


In [3]:
! pip freeze | grep vivarium

vivarium==3.6.6
vivarium_build_utils==2.1.2
vivarium_cluster_tools==2.2.2
vivarium_dependencies==1.0.4
-e git+https://github.com/ihmeuw/vivarium_gates_mncnh.git@2d0474bcba9673d9c3d4b89358fd34b2e86ef9de#egg=vivarium_gates_mncnh
vivarium_public_health==4.13.19
vivarium_testing_utils==0.3.1


In [4]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from vivarium import InteractiveContext, Artifact

In [5]:
import vivarium_gates_mncnh
from vivarium.framework.configuration import build_model_specification
from pathlib import Path

path = Path(vivarium_gates_mncnh.__file__).parent / 'model_specifications/model_spec.yaml'
custom_model_specification = build_model_specification(path)
del custom_model_specification.configuration.observers
# custom_model_specification.configuration.input_data.input_draw_number = 60
custom_model_specification.configuration.population.population_size = 20_000 * 10

artifact_path = custom_model_specification.configuration.input_data.artifact_path
art = Artifact(artifact_path)

artifact_path

'/mnt/team/simulation_science/pub/models/vivarium_gates_mncnh/artifacts/model27.1/ethiopia.hdf'

In [6]:
draw_num = custom_model_specification.configuration.input_data.input_draw_number
draw = 'draw_' + str(draw_num)
draw

'draw_60'

In [7]:
sim = InteractiveContext(custom_model_specification)

/mnt/share/homes/zmbc/mambaforge/envs/vivarium_gates_mncnh_simulation/lib/python3.11/site-packages/vivarium_public_health/plugins/parser.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


2026-01-22 16:11:35.284 | INFO     | simulation_1-artifact_manager:79 - Running simulation from artifact located at /mnt/team/simulation_science/pub/models/vivarium_gates_mncnh/artifacts/model27.1/ethiopia.hdf.


2026-01-22 16:11:35.284 | INFO     | simulation_1-artifact_manager:80 - Artifact base filter terms are ['draw == 60'].


2026-01-22 16:11:35.286 | INFO     | simulation_1-artifact_manager:81 - Artifact additional filter terms are None.


2026-01-22 16:11:58.221 | WARNING  | simulation_1-results_manager:446 - Specified excluded stratifications are already not included by default: ['stillbirth', 'partial_term']


2026-01-22 16:11:58.222 | WARNING  | simulation_1-results_manager:446 - Specified excluded stratifications are already not included by default: ['stillbirth', 'partial_term']


2026-01-22 16:11:58.344 | INFO     | simulation_1-results_context:129 - The following stratifications are registered but not used by any observers: 
['ferritin_screening_coverage', 'hemoglobin_screening_coverage', 'sex']


In [8]:
sim_step_name = sim.list_components()['risk_factor.hemoglobin']._sim_step_name
sim_step_name()

'first_trimester_anc'

In [9]:
def get_pop_with_pregnancy_duration(sim):
    pop = sim.get_population()
    return pd.concat([
        pop,
        sim.get_value('pregnancy_duration')(pop.index)
    ], axis=1)[['pregnancy_outcome', 'gestational_age_exposure', 'pregnancy_duration']]

In [10]:
pop = get_pop_with_pregnancy_duration(sim)
pregnancy_duration = pop.pregnancy_duration
assert ((pop.pregnancy_outcome == 'partial_term') == pregnancy_duration.notnull()).all(), "pregnancy duration not defined in exactly abortion/miscarriage/ectopic pregnancies pre-ultrasound"
# https://vivarium-research.readthedocs.io/en/latest/models/other_models/pregnancy/gbd_2021_mncnh/index.html#assign-gestational-age-at-end-of-pregnancy-abortion-miscarriage-ectopic-pregnancies
assert (pop[pop.pregnancy_outcome == 'partial_term'].pregnancy_duration >= pd.Timedelta(days=(6 * 7))).all(), "abortion/miscarriage/ectopic pregnancies with duration less than 6 weeks"
assert (pop[pop.pregnancy_outcome == 'partial_term'].pregnancy_duration < pd.Timedelta(days=(24 * 7))).all(), "abortion/miscarriage/ectopic pregnancies with duration greater than 24 weeks"

In [11]:
while sim_step_name() != "ultrasound":
    pop = get_pop_with_pregnancy_duration(sim)
    assert pop.gestational_age_exposure.isnull().all(), "gestational age assigned pre-ultrasound"
    assert pop.pregnancy_duration.equals(pregnancy_duration), "pregnancy duration changed pre-ultrasound"
    sim.step()

# Go past ultrasound step
sim.step()

2026-01-22 16:12:05.435 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2025-01-01 00:00:00


2026-01-22 16:12:17.268 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2025-01-02 00:00:00


2026-01-22 16:12:23.828 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2025-01-03 00:00:00


2026-01-22 16:12:33.959 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2025-01-04 00:00:00


In [12]:
pop = get_pop_with_pregnancy_duration(sim)
pop

,pregnancy_outcome,gestational_age_exposure,pregnancy_duration
0,live_birth,39.427973,275 days 23:53:58.342935390
1,partial_term,41.253371,126 days 23:31:15.581246692
2,partial_term,38.746684,110 days 17:55:58.891360316
3,partial_term,39.626390,153 days 06:54:42.088881381
4,live_birth,39.502264,276 days 12:22:49.012103490
...,...,...,...
199995,partial_term,37.342206,79 days 13:30:02.599728428
199996,live_birth,38.392324,268 days 17:54:37.254112934
199997,live_birth,41.163854,288 days 03:31:39.186681640
199998,live_birth,38.651817,270 days 13:30:18.952416953


In [13]:
assert pop.pregnancy_duration[pop.pregnancy_outcome == 'partial_term'].equals(pregnancy_duration[pop.pregnancy_outcome == 'partial_term']), "pregnancy duration changed for abortion/miscarriage/ectopic pregnancies"

In [14]:
assert np.allclose(
    pop[pop.pregnancy_outcome != 'partial_term'].pregnancy_duration / pd.Timedelta(days=7),
    pop[pop.pregnancy_outcome != 'partial_term'].gestational_age_exposure
), "pregnancy duration and gestational age disagree!"

In [15]:
assert (
    sim.get_value('gestational_age.birth_exposure')(pop.index)
    ==
    pop.gestational_age_exposure
).all(), "gestational age disagrees between value pipeline and state table!"

In [16]:
(pop[pop.pregnancy_outcome == 'live_birth'].pregnancy_duration / pd.Timedelta(days=7)).describe()

count    109556.000000
mean         38.061479
std           2.376997
min          19.937262
25%          37.324547
50%          38.353509
75%          39.488046
max          42.077509
Name: pregnancy_duration, dtype: float64

In [17]:
(pop[pop.pregnancy_outcome == 'stillbirth'].pregnancy_duration / pd.Timedelta(days=7)).describe()

count    3125.000000
mean       38.125167
std         2.274188
min        24.201812
25%        37.345561
50%        38.377843
75%        39.474093
max        42.067469
Name: pregnancy_duration, dtype: float64

In [18]:
# The 'risk specific shift' is the amount to delete
# https://github.com/ihmeuw/vivarium_gates_mncnh/blob/bdffaa88cb94eaa53a97df75be2e307dfd52c25e/src/vivarium_gates_mncnh/data/loader.py#L1326-L1335
ifa_ga_deletion = art.load('risk_factor.iron_folic_acid_supplementation.risk_specific_shift')
assert (ifa_ga_deletion.groupby('affected_entity').nunique() == 1).all().all()
ifa_ga_deletion = ifa_ga_deletion.groupby('affected_entity').first().loc['gestational_age'][draw]
ifa_ga_deletion

0.06667810244516607

In [19]:
# https://vivarium-research.readthedocs.io/en/latest/models/risk_exposures/low_birthweight_short_gestation/gbd_2021/index.html#converting-gbd-s-categorical-exposure-distribution-to-a-continuous-exposure-distribution
# NOTE: This minimum is on the LBWSG category sampled, but baseline IFA deletion decreases GA
assert (pop[pop.pregnancy_outcome == 'live_birth'].pregnancy_duration >= pd.Timedelta(days=((20 - ifa_ga_deletion) * 7))).all(), "live birth pregnancies with duration less than 20 weeks minus baseline IFA deletion"


In [20]:
# https://vivarium-research.readthedocs.io/en/latest/models/other_models/pregnancy/gbd_2021_mncnh/index.html#assign-birthweight-and-gestational-age-at-end-of-pregnancy-live-births-and-stillbirths
assert (pop[pop.pregnancy_outcome == 'stillbirth'].pregnancy_duration >= pd.Timedelta(days=((24 - ifa_ga_deletion) * 7))).all(), "stillbirth pregnancies with duration less than 24 weeks minus baseline IFA deletion"